# Cleaning and Visualizing GPX Files with Python 

During my gap year, I brought along a Garmin GPS receiver to log the places I travelled. I used it to track whenever I was moving from place to place and to log any special tours. 

This data was exported into my computer at random intervals as `.gpx` files. This notebook is a way to explore this data and hopefully glean some interesting insights from it.  

Note that these analyses can also be done via a desktop GIS, such as  QGIS, but I was interested in how this data can be calculated using Python. 

## Cleaning and sorting the data

First is to load the gpx files into memory. Using the `gpxpy` library, the files can be opened and parsed into a pandas DataFrame (note that it can also be loaded into a pandas GeoDataFrame, but for now I want to make use of the DataFrame `drop_duplicates()` function later on):

In [1]:
from os import listdir
from os.path import join, isfile

import gpxpy
import pandas as pd

path = '/media/hana/EXCHANGE/PROJECTS/GARMIN/_RAW/GPX'
files = listdir(path)
files = [join(path, f) for f in files if isfile(join(path, f))]

points = []
for f in files:
    #print(f)
    gpx_file = open(f, 'r')
    gpx = gpxpy.parse(gpx_file)

    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                points.append([point.time, point.latitude, point.longitude, point.elevation])

cols = ['Time', 'Lat', 'Lon', 'Elev']
points_df = pd.DataFrame(points, columns=cols)

print(points_df.info())
print(points_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21728 entries, 0 to 21727
Data columns (total 4 columns):
Time    21728 non-null datetime64[ns, SimpleTZ("Z")]
Lat     21728 non-null float64
Lon     21728 non-null float64
Elev    21728 non-null float64
dtypes: datetime64[ns, SimpleTZ("Z")](1), float64(3)
memory usage: 679.1 KB
None
                       Time        Lat         Lon     Elev
0 2016-08-17 10:56:27+00:00  47.909313  106.913609  1309.93
1 2016-08-17 11:06:27+00:00  47.882058  106.859710  1294.07
2 2016-08-17 11:16:27+00:00  47.852970  106.762846  1271.00
3 2016-08-17 13:37:48+00:00  47.851780  106.763506  1312.34
4 2016-08-17 13:41:37+00:00  47.852254  106.763606  1274.85


As seen above, there are 21728 points from all of the gpx files. 

Because of the the way the files were saved, some points/tracks are duplicated and overlap each other. Pandas makes this easy to deal with with `drop_duplicates()`. 

Specifically, rows with duplicate timestamps can be removed since all the files are recorded in the same timezone (UTC).  

After they are removed, the points can then be sorted by timestamp: 

In [2]:
#Remove duplicate points based on timestamp 
points_df.drop_duplicates(subset='Time', inplace=True)

#Sort by time
points_df.sort_values(by='Time', inplace=True)

print(points_df.head())
print(points_df.tail())

                           Time        Lat         Lon     Elev
14513 2016-04-24 00:46:28+00:00  50.964602 -114.052885  1065.76
14514 2016-04-24 00:46:51+00:00  50.964396 -114.053234  1055.67
14515 2016-04-24 00:47:21+00:00  50.964361 -114.053241  1047.97
14516 2016-04-24 00:47:49+00:00  50.964348 -114.053206  1041.25
14517 2016-04-24 00:48:19+00:00  50.964376 -114.053205  1044.61
                           Time        Lat       Lon   Elev
10077 2017-07-27 17:43:13+00:00  54.598049 -5.930262  17.44
10078 2017-07-27 17:43:21+00:00  54.597654 -5.930183  16.96
10079 2017-07-27 17:43:30+00:00  54.597566 -5.930158  17.44
10080 2017-07-27 17:43:45+00:00  54.597562 -5.930153  17.92
10081 2017-07-27 17:43:48+00:00  54.597561 -5.930155  18.40


You can see from the lat and long at the beginning and end of the file that I started in western Canada and ended around Ireland, which is what is expected. 

The points can also be plotted with Folium to see if they make sense:

In [3]:
import folium
pointlist = list(zip(points_df.Lat, points_df.Lon))

map0 = folium.Map(location=[0,0], zoom_start=1)
folium.PolyLine(pointlist,color='red', weight=2, opacity=1).add_to(map0)
map0 

As you can see from the track, I started from Canada and moved eastward towards Asia. At the end it shows going back to Europe when I moved to Ireland.

There are a few extra lines to and from Hong Kong, but that was because I visited the city more than once.

Now I can convert the points into a GeoDataFrame to make use of the spatial functionality:

In [4]:
import geopandas as gpd
from shapely.geometry import Point

geom = [Point(xy) for xy in zip(points_df.Lon, points_df.Lat, points_df.Elev)]
points_df.drop(['Lat', 'Lon'], axis=1, inplace=True)
crs = {'init': 'epsg:4326'}
points_gdf = gpd.GeoDataFrame(points_df, crs=crs, geometry=geom)

print(points_gdf.info())
#print(points_gdf.head())


<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 20732 entries, 14513 to 10081
Data columns (total 3 columns):
Time        20732 non-null datetime64[ns, SimpleTZ("Z")]
Elev        20732 non-null float64
geometry    20732 non-null object
dtypes: datetime64[ns, SimpleTZ("Z")](1), float64(1), object(1)
memory usage: 647.9+ KB
None


As shown above, there are now 20732 points in the dataset after the duplicates were removed. Each point now also has a geometry feature to play around with. 

## Calculating basic statistics 

Now that the data is cleaned and sorted, what kind of information can be taken from it? 

We can start with the distance travelled. This can be calculated directly from the latitude and longitude using `geopy`'s geodesic distance formula. 

For example, this is the distance in kilometres from the equator to the North Pole at the Prime Meridian:

In [5]:
from geopy.distance import geodesic

p0 = (0,0)
p1 = (90,0)
print(geodesic(p0, p1).meters/1000)

10001.965729312722


Using this formula would involve iterating through the dataframe, so I created a function that would also take each pair of subsequent points and create a LineString between them. Then I saved the LineString and distance into a GeoDataFrame. 

This means that the data would be stored in a Point and LineString dataframe, which could be handy for future calculations. 

In [6]:
from shapely.geometry import LineString

def generate_lines_from_points(pt_gdf):
    ls_geom = []
    ls_data = [] #[Start Time, End Time, Start Elev, End Elev, Dist, geometry] 
    print('Iterating through points...')
    for i in range(0, pt_gdf.shape[0]-1):
        p0 = pt_gdf.iloc[i]
        p1 = pt_gdf.iloc[i+1]

        p0_geom = p0['geometry']
        p1_geom = p1['geometry']

        p0_point = (p0_geom.y, p0_geom.x, p0_geom.z)
        p1_point = (p1_geom.y, p1_geom.x, p1_geom.z)
        dist = geodesic(p0_point, p1_point).meters

        line_data = [p0['Time'], p1['Time'], p0['Elev'], p1['Elev'], dist]

        p0_point = (p0_geom.x, p0_geom.y, p0_geom.z)
        p1_point = (p1_geom.x, p1_geom.y, p1_geom.z)
        ls_data.append(line_data)        
        ls_geom.append(LineString([p0_point, p1_point]))

    print('Generating geodataframe...')
    cols = ['Start Time', 'End Time', 'Start Elev', 'End Elev', 'Dist']
    line_df = pd.DataFrame(ls_data, columns=cols)
    line_gdf = gpd.GeoDataFrame(line_df, crs=pt_gdf.crs, geometry=ls_geom)

    return line_gdf

line_gdf = generate_lines_from_points(points_gdf)
print(line_gdf.info())
#print(line_gdf.head())


Iterating through points...
Generating geodataframe...
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 20731 entries, 0 to 20730
Data columns (total 6 columns):
Start Time    20731 non-null datetime64[ns, SimpleTZ("Z")]
End Time      20731 non-null datetime64[ns, SimpleTZ("Z")]
Start Elev    20731 non-null float64
End Elev      20731 non-null float64
Dist          20731 non-null float64
geometry      20731 non-null object
dtypes: datetime64[ns, SimpleTZ("Z")](2), float64(3), object(1)
memory usage: 971.9+ KB
None


Now, we can see the total distance travelled in kilometres by summing the distance column and dividing by 1000. 

In [7]:
line_gdf['Dist'].sum()/1000

49719.49880445211

## Analyzing country data

Now we can have a closer look at the countries that were visited. 

A shapefile of countries is loaded in GeoPandas (the data was downloaded from the 1:10m [Natural Earth dataset](https://www.naturalearthdata.com/downloads/50m-cultural-vectors/50m-admin-0-countries-2/)): 

In [8]:
countries = gpd.read_file('/media/hana/EXCHANGE/PROJECTS/GARMIN/countries/ne_10m_admin_0_countries.shp')
countries.plot(); 

Now we can use a spatial join with the country and points dataset to see which countries I had visited:

In [9]:
points_countries = gpd.sjoin(points_gdf, countries[['NAME','geometry']], how='left', op='intersects')
countries_visited = points_countries['NAME'].unique()

print(countries_visited)
print(countries_visited.size)

['Canada' 'Netherlands' 'Luxembourg' 'France' 'Switzerland'
 'United Kingdom' nan 'Belgium' 'Germany' 'Czechia' 'Poland' 'Ukraine'
 'Russia' 'Mongolia' 'China' 'Hong Kong' 'Vietnam' 'Thailand']
18


This shows that I'd visited 18 countries. Interestingly, however, it says I've been to Luxembourg, which isn't true. A quick query shows there is one datapoint for the country, and from the elevation, it was taken from a flight over the country! 

In [10]:
print(points_countries.loc[points_countries['NAME']=='Luxembourg', 'Elev'])

11375    9935.36
Name: Elev, dtype: float64


The `countries_visited` list also revealed countries with `nan` values. Interestingly, mapping them shows that they tend to be along coastlines; for example, here in Hong Kong, it seems some points are just outside the country polygon:

In [11]:
nan_points = points_countries.loc[points_countries['NAME'].isnull()]
#ax1 = countries.plot()
#nan_points.plot(ax=ax1, markersize=5)

map1 = folium.Map(location=[22.3192, 114.0170], zoom_start=12)
nan_points.apply(lambda row:folium.CircleMarker(location=[row.geometry.y, row.geometry.x], 
                                              radius=5)
                                             .add_to(map1), axis=1)
map1 

I will leave these as `nan` for now, but it is good to note in case I need to do other country calculations in the future. 